In [ ]:
from tensorflow import keras
import os, pickle, numpy
import numpy as np



# Import Train and Test datasets

In [ ]:
with open('train_qa.txt','rb') as file:
    train_data = pickle.load(file)
print('----------------------------------------------------------------------')
print(f'Train data len: {len(train_data)}')

with open('test_qa.txt','rb') as file:
    test_data = pickle.load(file)
print('----------------------------------------------------------------------')
print(f'Test data len: {len(test_data)}')
print('----------------------------------------------------------------------')

# Data structure

In [ ]:
x = 3
print(f'Set {x}')
print('----------------------------------------------------------------------')
print('Story:',' '.join(train_data[x-1][0]))
print('----------------------------------------------------------------------')
print('Question:',' '.join(train_data[x-1][1]))
print('----------------------------------------------------------------------')
print('Answer:',train_data[x-1][2])
print('----------------------------------------------------------------------')


# Create a vocabulary

This is unique for this particular dataset

In [ ]:
all_data = test_data + train_data
len(all_data)

In [ ]:
vocabulary = set()

# add unique words to vocabulary
for story, question, answer in all_data:
    vocabulary = vocabulary.union(set(story))
    vocabulary = vocabulary.union(set(question))

vocabulary.add('no')
vocabulary.add('yes')

vocab_size = len(vocabulary)+1 # +1 because in keras paddind function it is required to have a placeholder

print('Total number of unique words in questions and stories:',vocab_size-1)
vocabulary

# Check the longest story and longest question

In [ ]:
stories = []
questions = []

for story, question, answer in all_data:
    stories.append(len(story))
    questions.append(len(question))

max_story_len = max(stories)
max_question_len = max(questions)

print('Max story length is:', max_story_len,'words')
print('Max question length is:',max_question_len,'words')

# Tokenize data

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer


In [ ]:
tokenizer = Tokenizer(filters=[]) # because all sighs are important
tokenizer.fit_on_texts(vocabulary)
tokenizer.word_index

In [ ]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

def vectorize_data(data,
                    word_index,
                    max_story_len,
                    max_question_len):

    stories = []
    questions = []

    answers = []

    for story, question, answer in data:
        stories_part = [word_index[word.lower()] for word in story]         # return index of each word according to their position in word index for stories
        questions_part = [word_index[word.lower()] for word in question]    # return index of each word according to their position in word index for questions

        answers_part = np.zeros(len(word_index)+1)                          # placeholder
        answers_part[word_index[answer]] = 1                                # in the index position of 'yes' or 'no' put 1 

        stories.append(stories_part)
        questions.append(questions_part)
        answers.append(answers_part)

    return (pad_sequences(stories,maxlen=max_story_len),pad_sequences(questions,maxlen=max_question_len),np.array(answers))     # return padded data


    

# Create padded train and test data

In [ ]:
inputs_train, questions_train, answers_train = vectorize_data(train_data,word_index=tokenizer.word_index, max_story_len=max_story_len,max_question_len=max_question_len)
inputs_test, questions_test, answers_test = vectorize_data(test_data,word_index=tokenizer.word_index, max_story_len=max_story_len,max_question_len=max_question_len)

In [ ]:
print('inputs_train shape:',inputs_train.shape, 'questions_train shape:',questions_train.shape,'answers_train shape:', answers_train.shape)
print()
print('inputs_test shape:',inputs_test.shape, 'questions_test shape:',questions_test.shape,'answers_test shape:', answers_test.shape)

# Import and instatiate model

In [ ]:
from memory_network import memory_network

model = memory_network(max_story_len,max_question_len,vocab_size,optimizer='rmsprop',model_name='memory_network2')

# Create a model

In [ ]:
callbacks = [
            keras.callbacks.ModelCheckpoint(filepath=f'model_checkpoints/{model.name}.h5',save_best_only=True),
            keras.callbacks.ReduceLROnPlateau(monitor='val_loss', patience=10, factor=0.1, verbose=2, min_lr=1e-6),
            keras.callbacks.EarlyStopping(monitor='val_loss',patience=15)
            ]

# Training is surprisingly tricky part, remember not to train using GPU, model has to be trained in a sequential manner not parallel, so use CPU instead

In [ ]:
history = model.fit([inputs_train, questions_train], 
                    answers_train,
                    batch_size=32,
                    epochs=1200,
                    validation_data=([inputs_test, questions_test], 
                    answers_test),
                    callbacks = callbacks)

# Check what model has learned

In [26]:
def generate_question():
    import random

    x = random.randint(a=1,b=len(test_data))

    print('Story:',' '.join(test_data[x-1][0]))
    s = test_data[x-1][0]
    print('Question:',' '.join(test_data[x-1][1]))
    q = test_data[x-1][1]
    print('Answer:',test_data[x-1][2])
    a = test_data[x-1][2]

    return s, q, a, [(test_data[x-1])]


In [42]:
_,_,_,question = generate_question()

my_story,my_ques,my_ans = vectorize_data(question, tokenizer.word_index,max_story_len,max_question_len)

pred_results = model.predict(([ my_story, my_ques]))

yes_prob = pred_results[0][tokenizer.word_index['yes']]
no_prob = pred_results[0][tokenizer.word_index['no']]

if yes_prob > no_prob:
    k = 'yes'
    prob = yes_prob
else:
    k = 'no'
    prob = no_prob

print("Predicted answer is: ", k)
print("Probability of certainty was: ", round(prob*100,2),"%")

Story: Daniel went back to the kitchen . Mary grabbed the apple there . Daniel journeyed to the office . John went back to the office . Mary left the apple . Daniel went to the hallway . John went to the hallway . Daniel picked up the milk there .
Question: Is John in the kitchen ?
Answer: no
Predicted answer is:  no
Probability of certainty was:  99.88 %
